## Parameters

In [24]:
import math
from sage.rings.polynomial.polynomial_zmod_flint import Polynomial_zmod_flint as polynomial

# Kyber Parameters
q = 3329
k = 2
n = 256

RQ.<x> = GF(q)[]
RQ.gen()
f = x^(n+1) + 1

DEBUG = True

In [25]:
# Helper Functions
def reducePolynomials(matrx):
    cols = len(matrx.columns())
    rows = len(matrx.rows())
    out = [[matrx.coefficient((i,j)) for j in range(cols)] for i in range(rows)]
    for row in range(rows):
        for col in range(cols):
            # Divide the polynomial in out[row][col] by f, only keep the remainder
            _, rt = out[row][col].quo_rem(f)
            out[row][col] = rt
    return matrix(RQ, out)

def randomList(length, cbd=False):
    if cbd:
        # NOT real centered binomial distrbution!
        return [randrange(2) - randrange(2) for i in range(length)]
    else:
        return [randrange(q) for i in range(length)]

def randomPolyUniform(length):
    return RQ(randomList(length))

def randomPolyCbd(length):
    return RQ(randomList(length, cbd=True))

def compress(poly):
     q2 = math.ceil(q/2)
     return poly * q2

def decompress(poly) -> int:
    return [(1 if 3*(q/4) > Integer(i) > q/4 else 0) for i in poly]
    # return int(''.join([str(x) for x in dpoly]), 2)

def dbg(s: str = ''):
    if DEBUG:
        print(s)

## K-PKE KeyGen (INDCPA)

In [26]:
def kpke_keygen() -> (matrix, matrix, matrix):
    dbg('===== kpke_keygen =====')
    # A is a k*k dimension matrix
    A = []
    for _ in range(0, k):
        tA = []
        for _ in range(0, k):
            tA.append(randomPolyUniform(n))
        A.append(tA)
    A = matrix(A)
    dbg('A:')
    dbg(A)
    
    # s is a k*1 dimension matrix
    s = [[randomPolyCbd(n)] for _ in range(0, k)]
    s = matrix(s)
    dbg('s:')
    dbg(s)

    # e is a k*1 dimension matrix
    e = [[randomPolyCbd(n)] for _ in range(0, k)]
    e = matrix(e)
    dbg('e:')
    dbg(e)

    # compute t = A*s*e
    # t is a k*1 dimension matrix
    # Reduce
    t = reducePolynomials(A*s+e)
    dbg()

    return (A, t, s)

## K-PKE Encrypt (INDCPA)

In [27]:
def kpke_encrypt(A: matrix, t: matrix, m: int) -> (polynomial, polynomial):
    dbg('===== kpke_encrypt =====')
    # Ensure that m does not have more bits than n bits
    if len(m.bits()) > n:
        raise ValueError('m has more bits than n!')
    mm = m.bits()
    dbg('Bits of m:')
    dbg(mm)
    
    # We need m to be at least n bits long.
    # Pad mm with 0s until desired length is reached
    pad = [0 for _ in range(0, n - len(mm))]
    mm = RQ(mm + pad)
    dbg('Polynomial m:')
    dbg(mm)
    mm = compress(mm)
    dbg('Compressed m:')
    dbg(mm)

    # Generate r, e1, e2
    # r is a k*1 matrix
    r = [[randomPolyCbd(n)] for _ in range(0, k)]
    r = matrix(r)
    dbg('r:')
    dbg(r)

    # e1 is a k*1 matrix
    e1 = [[randomPolyCbd(n)] for _ in range(0, k)]
    e1 = matrix(e1)
    dbg('e1:')
    dbg(e1)

    # e2 is an n-length polynomial
    e2 = randomPolyCbd(n)
    dbg('e2:')
    dbg(e2)

    u = A.transpose() * r + e1
    v = t.transpose() * r + e2 + mm

    u = reducePolynomials(u)
    v = reducePolynomials(v)

    dbg('u:')
    dbg(u)
    dbg('v:')
    dbg(v)
    dbg()

    return (u, v)

    

## K-PKE Decryption

In [28]:
def kpke_decrypt(u: matrix, v: matrix, s: matrix) -> int:
    dbg('===== kpke_decrypt =====')
    # Compute a noisy result mn
    mn = v - s.transpose() * u
    mn = (reducePolynomials(mn)).coefficients()[0]
    dbg('Noisy recovered m:')
    dbg(mn)

    mn_c = mn.coefficients(sparse=false)
    mn_c.reverse()

    m_rec = decompress(mn_c)
    dbg('Decompressed m:')
    dbg(m_rec)

    m_rec = int(''.join([str(x) for x in m_rec]), 2)
    dbg('Recovered m:')
    dbg(m_rec)
    dbg()

    return m_rec

In [29]:
from os import urandom
m = Integer(int.from_bytes(urandom(((n+7) & (-8))//8), 'big'))
m &= 2**n-1


A, t, s = kpke_keygen()
u, v = kpke_encrypt(A, t, m)
mr = kpke_decrypt(u, v, s)
dbg('Original m:')
dbg(m)
dbg(m.bits())
if m != mr:
    raise ValueError('decrypted m does not match, final decompression likely failed')

===== kpke_keygen =====
A:
[              1111*x^255 + 627*x^254 + 2305*x^253 + 1227*x^252 + 3067*x^251 + 1424*x^250 + 490*x^249 + 1981*x^248 + 1711*x^247 + 1970*x^246 + 452*x^245 + 2886*x^244 + 1947*x^243 + 2529*x^242 + 2905*x^241 + 2670*x^240 + 15*x^239 + 1747*x^238 + 2945*x^237 + 2217*x^236 + 616*x^235 + 698*x^234 + 681*x^233 + 360*x^232 + 402*x^231 + 1029*x^230 + 2507*x^229 + 542*x^228 + 1161*x^227 + 59*x^226 + 501*x^225 + 1332*x^224 + 87*x^223 + 1787*x^222 + 206*x^221 + 1388*x^220 + 571*x^219 + 844*x^218 + 749*x^217 + 1390*x^216 + 1299*x^215 + 828*x^214 + 2800*x^213 + 1347*x^212 + 785*x^211 + 435*x^210 + 525*x^209 + 3286*x^208 + 2188*x^207 + 2717*x^206 + 994*x^205 + 2147*x^204 + 827*x^203 + 377*x^202 + 1149*x^201 + 2377*x^200 + 786*x^199 + 136*x^198 + 2655*x^197 + 659*x^196 + 2524*x^195 + 3235*x^194 + 1657*x^193 + 1805*x^192 + 2954*x^191 + 1294*x^190 + 1757*x^189 + 197*x^188 + 568*x^187 + 968*x^186 + 872*x^185 + 2408*x^184 + 713*x^183 + 1640*x^182 + 1054*x^181 + 2612*x^180 + 1773*